# Application of scVI on Cortex

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import os
import time
import warnings
warnings.filterwarnings('ignore')
from scvi.dataset import CortexDataset, RetinaDataset
from scvi.models import VAE
from scvi.inference import UnsupervisedTrainer, load_posterior
from scvi import set_seed
from scvi.dataset.csv import CsvDataset
from utils import *
np.random.seed(1234)
torch.manual_seed(1234)
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

## Clustering

### Load data and fit scVI

In [2]:
data_path = "/home/jzhaoaz/jiazhao/scPI_v2/package/datasets/cortex/"
expression_train = np.loadtxt(data_path + "data_train", dtype='float32')
label_train = np.loadtxt(data_path + "label_train")

save expression_train as .csv file named "expression_train.csv".

In [3]:
exp_train = CsvDataset(filename = data_path + "expression_train.csv")

[2020-07-15 13:24:49,689] INFO - scvi.dataset.csv | Preprocessing dataset
[2020-07-15 13:24:49,916] INFO - scvi.dataset.csv | Finished preprocessing dataset
[2020-07-15 13:24:49,922] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-07-15 13:24:49,923] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-07-15 13:24:49,951] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-07-15 13:24:49,953] INFO - scvi.dataset.dataset | Downsampled from 2253 to 2253 cells


In [4]:
n_epochs = 400
lr = 1e-3
use_cuda = True
n_celltype = 7
t_start = time.time()
vae = VAE(exp_train.nb_genes)
trainer = UnsupervisedTrainer(
    vae,
    exp_train,
    train_size=0.90,
    use_cuda=use_cuda,
    frequency=5,
)
trainer.train(n_epochs=n_epochs, lr=lr)
full = trainer.create_posterior(trainer.model, exp_train, indices=np.arange(len(exp_train)))
latent, batch_indices, labels = full.sequential().get_latent()
t_end = time.time()
print('Computational time = %1.1f minutes.' % ((t_end-t_start)/60.0))

[2020-07-15 13:24:52,904] INFO - scvi.inference.inference | KL warmup for 400 epochs



Computational time = 1.9 minutes.


### Evaluate clustering scores (ASW, NMI, ARI) and visualize latent distance

In [5]:
scores = cluster_scores(latent, 7, label_train)
print('Clustering scores [ASW, NMI, ARI] = [%f, %f, %f].' % (scores[0], scores[1], scores[2]))

Clustering scores [ASW, NMI, ARI] = [0.236863, 0.767987, 0.743195].


## Imputation

In [6]:
data_path = "/home/jzhaoaz/jiazhao/scPI_v2/package/datasets/cortex/"
X_true = np.loadtxt(data_path + "data_train", dtype='float32')
X_zero = np.loadtxt(data_path + "X_zero.txt", dtype='float32')
i = np.load(data_path + "i.npy")
j = np.load(data_path + "j.npy")
ix = np.load(data_path + "ix.npy")

save X_zero as .csv file named "X_zero.csv".

In [7]:
data_path = "/home/jzhaoaz/jiazhao/scPI_v2/package/datasets/cortex/"
exp_train = CsvDataset(filename = data_path + "X_zero.csv")

[2020-07-15 13:26:45,877] INFO - scvi.dataset.csv | Preprocessing dataset
[2020-07-15 13:26:46,119] INFO - scvi.dataset.csv | Finished preprocessing dataset
[2020-07-15 13:26:46,123] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-07-15 13:26:46,125] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-07-15 13:26:46,153] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-07-15 13:26:46,156] INFO - scvi.dataset.dataset | Downsampled from 2253 to 2253 cells


In [8]:
t_start = time.time()
vae = VAE(exp_train.nb_genes)
trainer = UnsupervisedTrainer(
    vae,
    exp_train,
    train_size=0.90,
    use_cuda=use_cuda,
    frequency=5,
)
trainer.train(n_epochs=n_epochs, lr=lr)
full = trainer.create_posterior(trainer.model, exp_train, indices=np.arange(len(exp_train)))
latent, batch_indices, labels = full.sequential().get_latent()
t_end = time.time()
print('Computational time = %1.1f minutes.' % ((t_end-t_start)/60.0))

[2020-07-15 13:26:46,225] INFO - scvi.inference.inference | KL warmup for 400 epochs



Computational time = 1.9 minutes.


### Evaluate imputation error

In [9]:
imputed_values = full.sequential().imputation()
impu_err = imputation_error(imputed_values, X_true, X_zero, i, j, ix)
print('Imputation error on randomly dropout data = %f.' % impu_err)

Imputation error on randomly dropout data = 2.328291.
